In [ ]:
!pip install jupyter_contrib_nbextensions #instalando nb-extensions

#importando bibliotecas
import pandas as pd
import numpy as np

#importando csv pro collab
from google.colab import files
uploaded = files.upload()

In [ ]:
#lendo o arquivo csv e definindo uma váriavel pra ele
import io
data = pd.read_csv(io.BytesIO(uploaded['kc_house_data.csv']))

In [ ]:
#verificando as dimensoes do meu dataset

print("Number of rows: ",data.shape[0]) 
print("Number of columns:",data.shape[1])

Number of rows:  21613
Number of columns: 21


In [ ]:
#verificando os tipo de dado de cada coluna
data.dtypes

In [ ]:
#transformando a coluna data para o tipo date
data['date'] = pd.to_datetime(data['date'])

In [ ]:
#arrumando as notações cientificas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
#descriptive statistics - selecionando apenas os tipos que são INT ou FLOAT para tirar a média deles
sub_attributes = data.select_dtypes(include=['int64','float64'])

#central tendency - mean, median
mean = pd.DataFrame(sub_attributes.apply(np.mean))
median = pd.DataFrame(sub_attributes.apply(np.median))

#dispersions - std (desvio padrao), minimo, maximo

std = pd.DataFrame(sub_attributes.apply(np.std))
min_ = pd.DataFrame(sub_attributes.apply(np.min))
max_ = pd.DataFrame(sub_attributes.apply(np.max))

df1 = pd.concat([max_,min_,mean, median,std], axis=1).reset_index()
df1.columns = ['attributes', 'maximo', 'minimo', 'media', 'mediana', 'std'] 

df1

In [ ]:
#dimensoes do novo dataframe
print("Number of rows: ",df1.shape[0]) 
print("Number of columns:",df1.shape[1])

Number of rows:  20
Number of columns: 6


In [ ]:
# 1 Crie uma nova coluna 'dormitory_type', onde: bedrooms = 1 (studio), bedrooms = 2 (apartament), bedrooms > (house)
data['dormitory_type'] = data['bedrooms'].apply( lambda x: 'studio' if x == 1 else
                                                           'apartment' if x == 2 else
                                                           'house' if x > 2 else 'NA')

In [ ]:
# LAÇO DE REPETIÇÃO - FOR  #criando uma nova coluna para os niveis de preço
# é necessário conhecer o tamanho do laço

for i in range(len(data)):
    if (data.loc[i, 'price'] >= 0) & (data.loc[i, 'price'] < 321950):
      data.loc[i, 'niveis'] = 'nivel_0'

    elif (data.loc[i, 'price'] >= 321950) & (data.loc[i, 'price'] < 450000):
      data.loc[i, 'niveis'] = 'nivel_1'

    elif (data.loc[i, 'price'] >= 450000) & (data.loc[i, 'price'] < 645000):
      data.loc[i, 'niveis'] = 'nivel_2'

    else:
      data.loc[i, 'niveis'] = 'nivel_3'

In [ ]:
#### UTILIZANDO A API GEOPY   

from geopy.geocoders import Nominatim  # improtanto library 

geolocator = Nominatim(user_agent='geoapiExercises')

data = data.head(100) #utilizando uma amostra de 100 linhas pra agilizar o processo

for i in range(len(data)):
  print('Loop: {}/{}'.format(i, len(data))) ##### PRINT PRA IR ATUALIZANDO SOBRE O LOOP -- #DICA AO USAR CHAVES ELE PEGA OS VALORS DE FORA DA STRING
  
  coordinates = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
  
  response = geolocator.reverse(coordinates) #API request - retorna um JSON, geralmente um DICTONARY

  if 'house_number' in response.raw['address']:                                                   # o if é para caso a celula nao contenha a informaçao requerida, ele nao retornar nada e nao der erro.
        data.loc[i, 'house_number'] = response.raw['address']['house_number']          # se tiver ele popula a coluna, caso nao, mantem a celula vazia.
  if 'road' in response.raw['address']:  
        data.loc[i, 'road'] = response.raw['address']['road']



import plotly.express as px

house = data[['id', 'lat', 'long', 'price', 'niveis']].copy()

fig1 = px.scatter_mapbox(house,
                         lat='lat',
                         lon='long',
                         size='price',
                         color='niveis',
                         color_discrete_sequence=["red", "lime", "aqua", "gold"],
                         size_max=15,
                         zoom=10)

fig1.update_layout(mapbox_style = 'open-street-map')
fig1.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
fig1.show()

In [ ]:
# ==================
# Bibliotecas
# ==================
!pip install jupyter_contrib_nbextensions #instalando nb-extensions
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim  
import plotly.express as px
# ==================
# Functions
# ==================
# REQUISITOS DE UMA FUNÇÃO
# 1 NOME - RESPONSABILIDADE
# 2 PARAMETROS DE ENTRADA
# 3 DADOS DE SAIDA

#data type verification
def show_dtypes(data):   ####SOMENTE USAR VARIAVEIS QUE ESTÃO DECLARADAS NO CABEÇALHO DE FUNÇÃO
  print(data.dtypes)
  return None

#show dimensions
def show_dimensions(data):
  print("Number of rows: ",data.shape[0]) 
  print("Number of columns:",data.shape[1])
  return None

#collect geodata
def collect_geodata(data, cols):
  #### UTILIZANDO A API GEOPY   
  geolocator = Nominatim(user_agent='geoapiExercises')
  
  #utilizando uma amostra de 100 linhas pra agilizar o processo
  data = data.head(100) 

  #create empty
  data.loc[:, cols[0]] = 'NA'
  data.loc[:, cols[1]] = 'NA'

  for i in range(len(data)):
    print('Loop: {}/{}'.format(i, len(data))) ##### PRINT PRA IR ATUALIZANDO SOBRE O LOOP -- #DICA AO USAR CHAVES ELE PEGA OS VALORS DE FORA DA STRING
    
    coordinates = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
    
    response = geolocator.reverse(coordinates) #API request - retorna um JSON, geralmente um DICTONARY

    if 'house_number' in response.raw['address']:                                                   # o if é para caso a celula nao contenha a informaçao requerida, ele nao retornar nada e nao der erro.
          data.loc[i, 'house_number'] = response.raw['address']['house_number']          # se tiver ele popula a coluna, caso nao, mantem a celula vazia.
    if 'road' in response.raw['address']:  
          data.loc[i, 'road'] = response.raw['address']['road']

    return data

### EXTRAÇÃO

#load dataset
from google.colab import files
uploaded = files.upload()
#reading and assigning data to a variable
import io
data = pd.read_csv(io.BytesIO(uploaded['kc_house_data.csv']))

# Extraction Analysis
# Dimensions
show_dimensions(data)
# Types
show_dtypes(data)
# ###TRANSFORMAÇÃO

#convert objecto to date
data['date'] = pd.to_datetime(data['date'])
#arrumando as notações cientificas
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#descriptive statistics - selecionando apenas os tipos que são INT ou FLOAT para tirar a média deles
sub_attributes = data.select_dtypes(include=['int64','float64'])

#central tendency - mean, median
mean = pd.DataFrame(sub_attributes.apply(np.mean))
median = pd.DataFrame(sub_attributes.apply(np.median))

#dispersions - std (desvio padrao), minimo, maximo

std = pd.DataFrame(sub_attributes.apply(np.std))
min_ = pd.DataFrame(sub_attributes.apply(np.min))
max_ = pd.DataFrame(sub_attributes.apply(np.max))

df1 = pd.concat([max_,min_,mean, median,std], axis=1).reset_index()
df1.columns = ['attributes', 'maximo', 'minimo', 'media', 'mediana', 'std'] 

show_dimensions(df1)

# # 1 Crie uma nova coluna 'dormitory_type', onde: bedrooms = 1 (studio), bedrooms = 2 (apartament), bedrooms > (house)
# data['dormitory_type'] = data['bedrooms'].apply( lambda x: 'studio' if x == 1 else
#                                                            'apartment' if x == 2 else
#                                                            'house' if x > 2 else 'NA')

# # LAÇO DE REPETIÇÃO - FOR  #criando uma nova coluna para os niveis de preço
# # é necessário conhecer o tamanho do laço

# for i in range(len(data)):
#     if (data.loc[i, 'price'] >= 0) & (data.loc[i, 'price'] < 321950):
#       data.loc[i, 'niveis'] = 'nivel_0'

#     elif (data.loc[i, 'price'] >= 321950) & (data.loc[i, 'price'] < 450000):
#       data.loc[i, 'niveis'] = 'nivel_1'

#     elif (data.loc[i, 'price'] >= 450000) & (data.loc[i, 'price'] < 645000):
#       data.loc[i, 'niveis'] = 'nivel_2'

#     else:
#       data.loc[i, 'niveis'] = 'nivel_3'

show_dimensions

# #---------CARGA
# ###MAP
# import plotly.express as px

# house = data[['id', 'lat', 'long', 'price', 'niveis']].copy()

# fig1 = px.scatter_mapbox(house,
#                          lat='lat',
#                          lon='long',
#                          size='price',
#                          color='niveis',
#                          color_discrete_sequence=["red", "lime", "aqua", "gold"],
#                          size_max=15,
#                          zoom=10)

# fig1.update_layout(mapbox_style = 'open-street-map')
# fig1.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
# fig1.show()

In [ ]:

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import plotly.express as px
# ==================
# Functions
# ==================
def show_dimensions( data ):
  print( 'Number of Rows:{}'.format( data.shape[0] ) )
  print( 'Number of Columns {}'.format( data.shape[1] ) )
  return None
def descriptive_statistics( data ):
# descriptive statistics
num_attributes = data.select_dtypes( include=['int64', 'float64'] )
# central tendency
media = pd.DataFrame( num_attributes.apply( np.mean ) )
mediana = pd.DataFrame( num_attributes.apply( np.median ) )
std = pd.DataFrame( num_attributes.apply( np.std ) )
# dispersion
std = pd.DataFrame( num_attributes.apply( np.std ) )
max_ = pd.DataFrame( num_attributes.apply( np.max ) )
min_ = pd.DataFrame( num_attributes.apply( np.min ) )
df1 = pd.concat( [max_, min_, media, mediana, std ], axis=1 ).reset_index()
df1.columns = ['attributes', 'mean', 'median', 'std', 'max', 'min']
return df1
